In [1]:
import pandas as pd
import json
import requests

In [2]:
bot_name = 'curekart'

In [3]:
# MAKE SURE RASA SERVER OF {bot_name} IS UP ON  http://localhost:5005/

In [4]:
def predict_node_name(text):
    data = {"text": text}
    url = 'http://localhost:5005/model/parse'
    response = requests.post(url, data=json.dumps(data))
    predicted_node = response.json()['intent']['name']
    predicted_node_score = response.json()['intent_ranking'][0]['confidence']
    return predicted_node, predicted_node_score

In [5]:
test_file_name = bot_name if '_subset' not in bot_name else bot_name.replace('_subset', '')
df_test = pd.read_csv(f'../../test/{test_file_name}_test.csv')
df_test.head()

,sentence,label
0,Order my product,ORDER_TAKING
1,Hy,NO_NODES_DETECTED
2,I want to order Wottle sensitive handwash refill,RECOMMEND_PRODUCT
3,have u started C D provision for pin code 702164,CHECK_PINCODE
4,How do I know it will deliver in my place,NO_NODES_DETECTED


In [6]:
df_test.shape

(991, 2)

In [7]:
predict_node_name('Order my product')

('ORDER_QUERY', 0.31236547231674194)

In [8]:
pred_nodes = []
pred_scores = []
for index, row in df_test.iterrows():
    node_name, node_score = predict_node_name(row['sentence'])
    pred_nodes.append(node_name)
    pred_scores.append(node_score)
df_test['predicted_node'] = pred_nodes
df_test['predicted_node_score'] = pred_scores
df_test.head()

,sentence,label,predicted_node,predicted_node_score
0,Order my product,ORDER_TAKING,ORDER_QUERY,0.312366
1,Hy,NO_NODES_DETECTED,RECOMMEND_PRODUCT,0.866968
2,I want to order Wottle sensitive handwash refill,RECOMMEND_PRODUCT,RECOMMEND_PRODUCT,0.452898
3,have u started C D provision for pin code 702164,CHECK_PINCODE,ORDER_STATUS,0.885070
4,How do I know it will deliver in my place,NO_NODES_DETECTED,RESUME_DELIVERY,0.942246


In [9]:
df_test.to_csv(f'../../preds/rasa_{bot_name}.csv', index=False)